In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml

from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, ARDRegression
from sklearn.cluster import KMeans

X = np.genfromtxt("X_train.txt", delimiter=None)
Y = np.genfromtxt("Y_train.txt", delimiter=None)
Xtest = np.genfromtxt("X_test.txt",delimiter=None)
X,Y = ml.shuffleData(X,Y)
Xtr, Xte, Ytr, Yte = ml.splitData(X, Y, 0.7)

In [ ]:
H = [1,2,3,4,5,6,10]
nnVMse = np.zeros(len(H))
nnTMse = np.zeros(len(H))

for i,h in enumerate(H):
    nn = MLPRegressor(hidden_layer_sizes=(h),solver="adam")
    nnpredict = nn.fit(Xtr, Ytr)
    nnTMse[i] = mean_squared_error(Ytr, nnpredict.predict(Xtr))
    nnVMse[i] = mean_squared_error(Yte, nnpredict.predict(Xte))

In [ ]:
for i in range(len(nnVMse)):
    print "h = {}: VMse = {:05f}, TMse = {:05f}".format(H[i],nnVMse[i],nnTMse[i])

In [6]:
clfs = [MLPRegressor(hidden_layer_sizes=(4),solver="adam"),
        GradientBoostingRegressor(loss='huber',max_depth=4,n_estimators=4000)
       ]

In [7]:
temp_results_t = np.zeros((  len(Ytr)        ,len(clfs) ))   #x1,x2
temp_results_v = np.zeros((  len(Yte)        ,len(clfs) ))   #y1,y2
test1        = np.zeros((  Xtest.shape[0] ,len(clfs) ))   
for i, clf in enumerate(clfs):
    clf.fit(Xtr,Ytr)
    temp_results_v[:,i] = clf.predict(Xte)
    temp_results_t[:,i] = clf.predict(Xtr)
    test1[:,i] = clf.predict(Xtest)

lr = LinearRegression(normalize=True)
lr.fit(temp_results_v, Yte)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [8]:
print "AUC on Train data: ",roc_auc_score(Ytr, lr.predict(temp_results_t))
print "AUC on Valid data: ",roc_auc_score(Yte, lr.predict(temp_results_v))

AUC on Train data:  0.820249024437
AUC on Valid data:  0.72466409965


In [20]:
Ypred_lr = lr.predict(test1)
np.savetxt('Yhat_nn_lr_3.txt', np.vstack( (np.arange(len(Ypred_lr)) , Ypred_lr) ).T, '%d, %.2f',header='ID,Prob1',comments='',delimiter=',');